In [64]:
import Pkg 
Pkg.activate(@__DIR__)
Pkg.resolve()
Pkg.instantiate()

import FiniteDiff
import ForwardDiff as FD
import Convex as cvx 
import ECOS
import MuJoCo as MJ

using LinearAlgebra
import Plots
using Random
using JLD2
using Test
using StaticArrays
using Printf
using Distributions
using MathOptInterface
using MuJoCo

  Activating project at `c:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\Project.toml`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\Manifest.toml`


In [65]:
Threads.nthreads()
install_visualiser()
init_visualiser()

   Resolving package versions...
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\Project.toml`
  No Changes to `C:\Users\Daniel\Documents\Python-Projects\ocrl-piano\single_finger_free_trajectories\Manifest.toml`


In [66]:
model = load_model("models/scene_right_piano_hand.xml")
data = init_data(model)
# reset!(model, data)
# visualise!(model, data)


MuJoCo Data object

In [67]:
fingertip = MJ.site(data, "fingertip_index")
fingertip.xpos

target = MJ.geom(data, "keymarker_1")
target.xpos

error = zeros(3)
println(error)
error = fingertip.xpos - target.xpos
println(error)

[0.0, 0.0, 0.0]
[0.0, 0.0, 0.0]


In [68]:
# function multifinger_IK_cost(m::Model, d::Data, neutral_pose::Vector, fingertip_targets::Vector)

# end

function multifinger_IK_cost(params::NamedTuple, x::Vector)::Real
    """
    Compute the cost of the IK problem for a given set of joint angles q and target names.
    """
    model = params.model
    data = params.data
    neutral_joint_angles = params.neutral_joint_angles
    fingertip_targets = params.fingertip_site_names
    qpos = params.neutral_joint_angles
    Q = params.Q

    cost = 0.0
    cost += 0.5*(x - neutral_joint_angles)'*Q*(x - neutral_joint_angles) # regularization cost

    data.qpos .= x
    # step!(model, data)

    return cost
end

function IK_equality_constraints(params::NamedTuple, x::Vector)::Vector
    """
    Compute the equality constraints for the IK problem for a given set of joint angles q and target names.
    """
    model = params.model
    data = params.data
    neutral_joint_angles = params.neutral_joint_angles
    fingertip_site_names = params.fingertip_site_names
    target_site_names = params.target_site_names
    neutral_joint_angles = params.neutral_joint_angles

    data.qpos .= x
    step!(model, data)

    @assert length(x) == length(params.neutral_joint_angles)
    @assert length(params.fingertip_site_names) == length(params.target_site_names)


    error = zeros(length(fingertip_site_names)*3) # positional error for fingertips 
    
    for i=1:length(fingertip_site_names)
        fingertip = MJ.site(data, fingertip_site_names[i])
        target = MJ.geom(data, target_site_names[i])
        
        # site_id = mj_name2id(model, mjOBJ_SITE, fingertip_site_names[i])
        # target_id = mj_name2id(model, mjOBJ_SITE, target_site_names[i])
        
        # preallocate arrays for a single site computation
        # error_pos = error[1+3*(i-1):3*i]
        # error_rot = error[6*i+3:6*(i+1)]
        
        # positional error
        error[1+3*(i-1):3*i] .= fingertip.xpos - target.xpos



        # # orientation error
        # site_quat = neros(4)
        # site_quat_conj = zeros(4)
        
        # target_quat = zeros(4)
        
        # error_quat = zeros(4)
        
        # mujoco.mju_mat2Quat(site_quat, data.site(site_id).xmat)
        # mujoco.mju_mat2Quat(target_quat, data.geom(target_id).xmat)
        
        # mujoco.mju_negQuat(site_quat_conj, site_quat)
        # mujoco.mju_mulQuat(error_quat, target_quat, site_quat_conj)
        # mujoco.mju_quat2Vel(error_rot, error_quat, 1.0)
    end

    return error
end

function IK_inequality_constraints(params::NamedTuple, x::Vector)::Vector
    return zeros(eltype(x), 0)
end


IK_inequality_constraints (generic function with 1 method)

In [69]:
include(joinpath(@__DIR__, "utils","fmincon.jl"))

fmincon (generic function with 1 method)

In [73]:
function solve_multifinger_IK(;verbose=true)
    # instantiate model and data
    model = load_model("models/scene_right_piano_hand.xml")
    data = init_data(model)
    reset!(model, data)

    # set the neutral pose
    neutral_joint_angles = vec(deepcopy(data.qpos))
    # neutral_joint_angles = convert(Vector{Float64}, neutral_joint_angles)
    
    # neutral_joint_angles = zeros(model.nq)
    @show typeof(neutral_joint_angles)

    # reset the model and data
    reset!(model, data)
    @show size(neutral_joint_angles)

    # LQR cost for neutral neutral_pose
    Q = 1e-3*diagm(ones(model.nq))
    @show size(Q)

    # fingertip sites and targets
    fingertip_site_names = ["fingertip_index", "fingertip_middle", "fingertip_ring"]
    target_site_names = ["keymarker_1", "keymarker_2", "keymarker_3"]

    params = (Q=Q, model=model, data=data, fingertip_site_names, target_site_names, neutral_joint_angles)

    # primal bounds
    x_l = model.jnt_range[:,1]
    x_u = model.jnt_range[:,2]

    # inequality constraints
    c_l = zeros(0)
    c_u = zeros(0)



    # initial_guess 
    z0=0.001*rand(model.nq)

    # multifinger_IK_cost(params, z0)
    # IK_equality_constraints(params, z0)
    diff_type = :finite
    Z = fmincon(multifinger_IK_cost, IK_equality_constraints, IK_inequality_constraints, x_l, x_u, c_l, c_u, z0, params, diff_type; tol=1e-6, c_tol=1e-4, max_iters = 50, verbose=verbose)
    return Z
end

solve_multifinger_IK (generic function with 1 method)

In [71]:
qpos = solve_multifinger_IK()

model = load_model("models/scene_right_piano_hand.xml")
data = init_data(model)
reset!(model, data)
data.qpos .= qpos


typeof(neutral_joint_angles) = Base.ReshapedArray{Float64, 1, Transpose{Float64, UnsafeArrays.UnsafeArray{Float64, 2}}, Tuple{Base.MultiplicativeInverses.SignedMultiplicativeInverse{Int64}}}
size(neutral_joint_angles) = (27,)
size(Q) = (27, 27)
---------checking dimensions of everything----------
---------all dimensions good------------------------
---------diff type set to :finite (FiniteDiff.jl)---
---------testing objective gradient-----------------
---------testing constraint Jacobian----------------
---------successfully compiled both derivatives-----
---------IPOPT beginning solve----------------------
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:      162
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:       27
                     variables with only lower bounds: 

27×1 transpose(::UnsafeArrays.UnsafeArray{Float64, 2}) with eltype Float64:
  0.14329295407205603
 -0.4403182872711473
  0.027643055975507553
 -0.11789671389476032
 -0.041386534392815574
 -0.034542496534784826
 -0.20352860268532885
  0.03531015397615159
  0.046311948525791645
  0.05473159558073913
  ⋮
  0.0012288399101613825
  0.033945168053773234
  0.05739165096736398
  0.057391650967364025
  0.008305259773519192
  0.05504795033413831
  0.0004037568897245432
  0.003398173510081537
  0.03418323766320522

In [72]:
visualise!(model, data)

 __  __            _        _____       _ _ 
|  \/  |          | |      / ____|     (_) |
| \  / |_   _     | | ___ | |     ___   _| |
| |\/| | | | |_   | |/ _ \| |    / _ \ | | |
| |  | | |_| | |__| | (_) | |___| (_) || | |
|_|  |_|\__,_|\____/ \___/ \_____\___(_) |_|
                                      _/ |  
                                     |__/   

Press "F1" to show the help message.
